In [ ]:
%load_ext autoreload
%autoreload 2
import Networks
from Networks.UNet3D import UNet3D
from Networks.UNet2D import UNet
from Networks.UNet2D_Attention import UNet2D_Attention
from Networks.UNet2D_Double_Blocks import UNet2D_Double_Blocks 
from Networks.UNet2D_Upsample import UNet2D_Upsample
from Networks.NestedUNet import NestedUNet
from LossFunctions import dice_loss
from Tester import predict
from DataHandler import getSplitData
from DataHandler import getTestingData
from DataHandler import numpy_dataset
from Trainer import Trainer
from torch.utils.data import DataLoader
import torch



In [ ]:
X_train, X_val, X_test, y_train, y_val, y_test = getSplitData(["Guys","HH"], dimensions = 2)

In [ ]:
#This is used to train a single model 
torch.cuda.empty_cache()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
net = UNet(in_channels=1, init_features=20, out_channels=5).to(device)

loss = dice_loss()
optim = torch.optim.Adam(net.parameters(), lr = 0.0005)
trainer = Trainer(net, loss, optim)

train_dataset = numpy_dataset(X_train, y_train)
val_dataset = numpy_dataset(X_val, y_val) 
train_dataloader = DataLoader(train_dataset, batch_size = 32, shuffle=True, drop_last = True)
val_dataloader = DataLoader(val_dataset, batch_size = 32, shuffle = False, drop_last = True)
net = trainer.do_training(train_dataloader, val_dataloader, max_epochs = 30)


In [ ]:
# load data as necessary
# test_dataset = numpy_dataset(X_test, y_test)
# test_dataloader = DataLoader(test_dataset)
# print(X_test.shape)
# # pred,true = predict(net,test_dataloader)

# print(pred.shape) 

X_IOP, y_IOP = getTestingData(["IOP"], dimensions = 3)
iop_dataset = numpy_dataset(X_IOP, y_IOP)
iop_dataloader = DataLoader(iop_dataset)
# pred_IOP, true_IOP = predict(net, iop_dataloader)
# print(pred_IOP.shape)



In [ ]:
# test using dice scores
from EvaluationMetrics import dice_score
#Using 2D 
print("pred/true dice scores.")
x = dice_score(pred,true)
print("iop pred/true dice scores:")
y = dice_score(pred_IOP, true_IOP)

In [ ]:
#This block is used for a grid search
feature_sizes = [64]
batch_sizes = [32,64] 

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
for batch_size in batch_sizes:
    for feature_size in feature_sizes:
        print("Testing feature_size: ", feature_size, " and batch_size: ", batch_size)
        torch.cuda.empty_cache()
        X_train, X_val, X_test, y_train, y_val, y_test = getSplitData(["HH","Guys"], dimensions = 2)

        net = UNet(in_channels=1, init_features=feature_size, out_channels=5).to(device)
        loss = dice_loss()
        optim = torch.optim.Adam(net.parameters(), lr = 0.0005)
        trainer = Trainer(net, loss, optim)
        #train_dataloader, val_dataloader, max_epochs

        train_dataset = numpy_dataset(X_train, y_train)
        val_dataset = numpy_dataset(X_val, y_val) 
        train_dataloader = DataLoader(train_dataset, batch_size = batch_size, shuffle=True, drop_last = True)
        val_dataloader = DataLoader(val_dataset, batch_size = batch_size, shuffle = False, drop_last = True)
        net = trainer.do_training(train_dataloader, val_dataloader, max_epochs = 45)

        torch.save(net, "unetsimple3depth experiment f_size " + str(feature_size) + " b_Size " + str(batch_size) + ".pt")

        test_dataset = numpy_dataset(X_test, y_test)
        test_dataloader = DataLoader(test_dataset)
        print(X_test.shape)
        pred,true = predict(net,test_dataloader)


        X_IOP, y_IOP = getTestingData(["IOP"], dimensions = 2)
        iop_dataset = numpy_dataset(X_IOP, y_IOP)
        iop_dataloader = DataLoader(iop_dataset)
        pred_IOP, true_IOP = predict(net, iop_dataloader)

        from EvaluationMetrics import dice_score
        #Using 2D 
        print("pred/true dice scores.")
        x = dice_score(pred,true)
        print("iop pred/true dice scores:")
        y = dice_score(pred_IOP, true_IOP)





In [ ]:
# Used to test for outliers
from EvaluationMetrics import dice_score
import matplotlib.pyplot as plt 
import numpy as np 
def outlierTest(pred,true, threshold):
    scores = []
    for i in range(len(pred)):
        score = np.mean(dice_score(pred[i], true[i], verbose = False))
        scores.append(score)
        if (score < threshold):
            print(i)
    plt.plot(scores)
    plt.xlabel("Average dice score")
    plt.show()
# print("2D Test")
# outlierTest(pred_2D, true_2D, threshold = 0.5)
# print("3D Test")
# # outlierTest(pred_3D, true_3D, threshold = 0.5)

In [ ]:
net_3d = torch.load('unet3d_softmax.pt')
net_3d.eval()
X, y = getTestingData(["HH","Guys","IOP"])
all_dataset= numpy_dataset(X,y)
all_dataloader = DataLoader(all_dataset,batch_size = 40, shuffle=False, drop_last = True) 

pred, true = predict(net_3d, all_dataloader)
print(pred.shape)
print(true.shape)
# outlierTest(pred, true)
# print(pred_IOP.shape)

In [ ]:
# Used for qualitative visualisation
for i in range(40):
    plt.imshow(pred[544][i])
    plt.show()
    plt.imshow(true[544][i])
    plt.show()
    plt.imshow(X[544][0][i])
    plt.show()
